In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
#dealing with mnist, 10 output channels requireed minm
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #28x28 (initially)
        #target rf ~ 28

        self.conv1 = nn.Sequential(  #26, rf = 3
            nn.Conv2d(1, 10, 3, padding=1, stride = 1, bias = False), #input -?(28) OUtput? = (28) RF = (3)
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.03)
        )

        self.extra1 = nn.Sequential(  #26, rf = 3
            nn.Conv2d(10, 16, 3, padding=1, stride = 1, bias = False), #input -?(28) OUtput? = (28) RF = (3)
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.03)
        )
        ### TRANSITION BLOCK
        self.tb = nn.Sequential( #14
            nn.Conv2d(16, 10, 1, padding = 0, stride = 1, bias = False) #size = 11 , rf = 8
        )
        self.pool1 = nn.MaxPool2d(2,2)

        self.conv2 = nn.Sequential(
            nn.Conv2d(10, 16, 3, padding = 0 , stride = 1, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.03)
            )

        self.conv3 = nn.Sequential( #size 3
            nn.Conv2d(16, 16, 3, padding = 0 , stride = 1, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.03)
        )
        self.pool2 = nn.MaxPool2d(2,2) #@ rf 10

        self.conv4 = nn.Sequential( #size 1
            nn.Conv2d(16, 16, 3, padding = 0 , stride = 1, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.03)
        )

        self.extra2 = nn.Sequential( #size 1
            nn.Conv2d(16, 16, 1, padding = 0 , stride = 1, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.03)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(16, 10, 1, padding = 0 , stride = 1, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.03)
        )

        self.adp = nn.AdaptiveAvgPool2d(1)

        self.conv6 = nn.Sequential(
            nn.Conv2d(10, 10, 1, padding = 0, stride = 1, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.03)
        )
        self.extra3 = nn.Sequential(
            nn.Conv2d(10, 10, 1, padding = 0, stride = 1, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.03)
        )


    def forward(self, x):

        x = self.conv1(x)
        x = self.extra1(x)
        x = self.tb(x)
        x = self.pool1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        x = self.pool2(x)
        x = self.conv4(x)
        x = self.extra2(x)
        x = self.conv5(x)
        # print("Conv5, ", x.shape)
        x = self.adp(x)
        x = self.conv6(x) #2 10 1 1
        x = self.extra3(x)
        # print("Conv6, ", x.shape)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)

#CB2
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]              90
              ReLU-2           [-1, 10, 28, 28]               0
       BatchNorm2d-3           [-1, 10, 28, 28]              20
           Dropout-4           [-1, 10, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           1,440
              ReLU-6           [-1, 16, 28, 28]               0
       BatchNorm2d-7           [-1, 16, 28, 28]              32
           Dropout-8           [-1, 16, 28, 28]               0
            Conv2d-9           [-1, 10, 28, 28]             160
        MaxPool2d-10           [-1, 10, 14, 14]               0
           Conv2d-11           [-1, 16, 12, 12]           1,440
             ReLU-12           [-1, 16, 12, 12]               0
      BatchNorm2d-13           [-1, 16, 12, 12]              32
          Dropout-14           [-1, 16,

<ipython-input-6-1947fbb0cd8d>:99: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
torch.manual_seed(1)
batch_size = 128


kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        # transforms.RandomRotation((-7.0, 7.0), fill=(1,)),  #adding iamge augmentation
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        y_pred = model(data)
        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    Accuracy = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return Accuracy

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience = 7)

for epoch in range(1, 16):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test_accuracy = test(model, device, test_loader)
    scheduler.step(test_accuracy)

EPOCH: 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-6-1947fbb0cd8d>:99: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Loss=0.17869937419891357 Batch_id=468 Accuracy=87.14: 100%|██████████| 469/469 [00:19<00:00, 23.46it/s]



Test set: Average loss: 0.1003, Accuracy: 9759/10000 (98%)

EPOCH: 2


Loss=0.12769021093845367 Batch_id=468 Accuracy=95.04: 100%|██████████| 469/469 [00:20<00:00, 23.22it/s]



Test set: Average loss: 0.0738, Accuracy: 9801/10000 (98%)

EPOCH: 3


...


Test set: Average loss: 0.0361, Accuracy: 9896/10000 (99%)

EPOCH: 14


Loss=0.12123358994722366 Batch_id=468 Accuracy=97.86: 100%|██████████| 469/469 [00:19<00:00, 24.10it/s]



Test set: Average loss: 0.0341, Accuracy: 9897/10000 (99%)

EPOCH: 15


Loss=0.05803273245692253 Batch_id=468 Accuracy=98.05: 100%|██████████| 469/469 [00:19<00:00, 24.38it/s]



Test set: Average loss: 0.0343, Accuracy: 9899/10000 (99%)

